In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import seaborn as sns
import collections

import warnings
warnings.filterwarnings('ignore')

# Utils
file_path = os.getcwd()
utils_path = os.path.abspath(os.path.join(file_path, '../src/data'))
sys.path.append(utils_path)
import utils_data

In [2]:
# Load agg response
# rsp = pd.read_csv(os.join.path(file_path, '../data/from_combined'))
rsp = pd.read_csv('/Users/apartin/work/jdacs/Benchmarks/Data/Pilot1/combined_single_response_agg', sep='\t')
rsp.insert(loc=2, column='CELL_abs', value=rsp['CELL'].map(lambda s: s.split('.')[1]))
print('rsp.shape', rsp.shape)
display(rsp[:2])

(4484081, 15)


,SOURCE,CELL,CELL_abs,DRUG,STUDY,AUC,IC50,EC50,EC50se,R2fit,Einf,HS,AAC1,AUC1,DSS1
0,CCLE,CCLE.1321N1,1321N1,CCLE.1,fake_exp,0.8330,4.883,5.183,0.5746,0.9864,0.2017,0.7463,0.1017,0.8983,0.0551
1,CCLE,CCLE.1321N1,1321N1,CCLE.10,fake_exp,0.7909,5.217,5.217,2.2840,0.6526,0.0000,0.8993,0.1136,0.8864,0.0728


In [18]:
# Load cell mapping
clmap = pd.read_csv('/Users/apartin/work/jdacs/Benchmarks/Data/Pilot1/cl_mapping', sep='\t', header=None)
clmap.rename(columns={0: 'from_cell', 1: 'to_cell'}, inplace=True)
print('clmap.shape', clmap.shape)
display(clmap[:5])

clmap.shape (1841, 2)


,from_cell,to_cell
0,CCLE.22RV1,GDSC.22RV1
1,CCLE.2313287,GDSC.23132-87
2,CCLE.42MGBA,GDSC.42-MG-BA
3,CCLE.5637,GDSC.5637
4,CCLE.639V,GDSC.639-V


**Note!** If we strip all the source prefixes from the cell names (e.g., `CCLE`, `GDSC`), there are cases of mismatch between cell names. For example: `CCLE.42MGBA` is mapped to `GDSC.42-MG-BA`. After stripping, `42MGBA` and `42-MG-BA` do not match. This is important because if we do some analysis on the aggregation of cell lines, these cell lines will be in the same group. For example, when I groupby the absolute cell line name (`CELL_abs`), and aggregate on `SOURCE`, it shows that `42MGBA` appears in while `CCLE` while `42-MG-BA` appears in `[CTRP, GDSC]`.

In [27]:
aa = rsp.groupby('CELL_abs').agg({'SOURCE': np.unique}).reset_index()
print('aa.shape', aa.shape)
display(aa[:10])

aa.shape (1894, 2)


,CELL_abs,SOURCE
0,-,GDSC
1,1321N1,"[CCLE, gCSI]"
2,2004,CTRP
3,201T,GDSC
4,22RV1,"[CCLE, CTRP, GDSC]"
5,23132-87,"[CTRP, GDSC, gCSI]"
6,253J,CTRP
7,253J-BV,CTRP
8,42-MG-BA,"[CTRP, GDSC]"
9,42MGBA,CCLE


Thus, we should consider to drop the relevant characters ...

In [26]:
rsp[rsp['SOURCE']=='CTRP']['CELL'].value_counts()[:10]

CTRP.A549         1401
CTRP.NCI-H1869     943
CTRP.KE-39         943
CTRP.DU-145        943
CTRP.CCF-STTG1     935
CTRP.NCI-H460      931
CTRP.NCI-H1299     930
CTRP.SUIT-2        929
CTRP.A375          927
CTRP.OAW-28        927
Name: CELL, dtype: int64

In [ ]:
clmap['']

In [19]:
aa.iloc[0,:]

CELL_abs       -
SOURCE      GDSC
Name: 0, dtype: object

In [28]:
rsp.loc[rsp['SOURCE']=='GDSC', 'CELL'].unique()
# rsp.iloc[rsp['SOURCE'].isin(['GDSC'])]

array(['GDSC.-', 'GDSC.201T', 'GDSC.22RV1', ..., 'GDSC.huH-1',
       'GDSC.no-10', 'GDSC.no-11'], dtype=object)

,CCLE.22RV1,GDSC.22RV1
0,CCLE.2313287,GDSC.23132-87
1,CCLE.42MGBA,GDSC.42-MG-BA
2,CCLE.5637,GDSC.5637
3,CCLE.639V,GDSC.639-V
4,CCLE.647V,GDSC.647-V
5,CCLE.697,GDSC.697
6,CCLE.769P,GDSC.769-P
7,CCLE.8MGBA,GDSC.8-MG-BA
8,CCLE.8305C,GDSC.8305C
9,CCLE.8505C,GDSC.8505C


## sandbox

In [29]:
from sklearn.model_selection import GroupKFold
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
y = np.array([1, 2, 3, 4])
groups = np.array([0, 0, 2, 2])

In [30]:
group_kfold = GroupKFold(n_splits=2)
group_kfold.get_n_splits(X, y, groups)

2

In [32]:
for train_index, test_index in group_kfold.split(X, y, groups):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    print(X_train, X_test, y_train, y_test)

TRAIN: [0 1] TEST: [2 3]
[[1 2]
 [3 4]] [[5 6]
 [7 8]] [1 2] [3 4]
TRAIN: [2 3] TEST: [0 1]
[[5 6]
 [7 8]] [[1 2]
 [3 4]] [3 4] [1 2]
